In [2]:
pip install tensorflowjs

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import print_function, division
import os
import ntpath
import numpy as np
from PIL import Image
from os.path import join, exists

# local libs
from suim_net import SUIM_Net
from data_utils import getPaths
from data_utils import processSUIMDataRFHW
from data_utils import getRobotFishHumanReefWrecks

C:\Users\marti\anaconda3\envs\env_deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\marti\anaconda3\envs\env_deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\marti\anaconda3\envs\env_deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\marti\anaconda3\envs\env_deeple

In [2]:
test_dir = "TEST/TEST/images/"
## sample and ckpt dir
samples_dir = "data/test/output/"
RO_dir = samples_dir + "RO/"
FB_dir = samples_dir + "FV/"
WR_dir = samples_dir + "WR/"
HD_dir = samples_dir + "HD/"
RI_dir = samples_dir + "RI/" 
if not exists(samples_dir): os.makedirs(samples_dir)
if not exists(RO_dir): os.makedirs(RO_dir)
if not exists(FB_dir): os.makedirs(FB_dir)
if not exists(WR_dir): os.makedirs(WR_dir)
if not exists(HD_dir): os.makedirs(HD_dir)
if not exists(RI_dir): os.makedirs(RI_dir)

In [3]:
## input/output shapes
base_ = 'RSB' # or 'VGG'
if base_=='RSB':
    im_res_ = (320, 240, 3) 
    ckpt_name = "suimnet_rsb5.hdf5"
else: 
    im_res_ = (320, 256, 3)
    ckpt_name = "suimnet_vgg5.hdf5"
suimnet = SUIM_Net(base=base_, im_res=im_res_, n_classes=5)
model = suimnet.model
print (model.summary())
model.load_weights(ckpt_name)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\marti\Desktop\projet m1\Untitled Folder\suim_net.py:143: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  return Model(input=img_input, output=out)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 236, 316, 64) 4864        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 236, 316, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 236, 316, 64) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [10]:
im_h, im_w = im_res_[1], im_res_[0]
def testGenerator():
    # test all images in the directory
    assert exists(test_dir), "local image path doesnt exist"
    imgs = []
    for p in getPaths(test_dir):
        # read and scale inputs
        img = Image.open(p).resize((im_w, im_h))
        img = np.array(img)/255.
        img = np.expand_dims(img, axis=0)
        # inference
        out_img = model.predict(img)
        # thresholding
        out_img[out_img>0.5] = 1.
        out_img[out_img<=0.5] = 0.
        print ("tested: {0}".format(p))
        # get filename
        img_name = ntpath.basename(p).split('.')[0] + '.bmp'
        # save individual output masks
        ROs = np.reshape(out_img[0,:,:,0], (im_h, im_w))
        FVs = np.reshape(out_img[0,:,:,1], (im_h, im_w))
        HDs = np.reshape(out_img[0,:,:,2], (im_h, im_w))
        RIs = np.reshape(out_img[0,:,:,3], (im_h, im_w))
        WRs = np.reshape(out_img[0,:,:,4], (im_h, im_w))
        Image.fromarray(np.uint8(ROs*255.)).save(RO_dir+img_name)
        Image.fromarray(np.uint8(FVs*255.)).save(FB_dir+img_name)
        Image.fromarray(np.uint8(HDs*255.)).save(HD_dir+img_name)
        Image.fromarray(np.uint8(RIs*255.)).save(RI_dir+img_name)
        Image.fromarray(np.uint8(WRs*255.)).save(WR_dir+img_name)

# test images
testGenerator()

tested: TEST/TEST/images/d_r_122_.jpg
tested: TEST/TEST/images/d_r_129_.jpg
tested: TEST/TEST/images/d_r_144_.jpg
tested: TEST/TEST/images/d_r_166_.jpg
tested: TEST/TEST/images/d_r_182_.jpg
tested: TEST/TEST/images/d_r_196_.jpg
tested: TEST/TEST/images/d_r_233_.jpg
tested: TEST/TEST/images/d_r_277_.jpg
tested: TEST/TEST/images/d_r_300_.jpg
tested: TEST/TEST/images/d_r_345_.jpg
tested: TEST/TEST/images/d_r_379_.jpg
tested: TEST/TEST/images/d_r_384_.jpg
tested: TEST/TEST/images/d_r_386_.jpg
tested: TEST/TEST/images/d_r_465_.jpg
tested: TEST/TEST/images/d_r_47_.jpg
tested: TEST/TEST/images/d_r_547_.jpg
tested: TEST/TEST/images/d_r_552_.jpg
tested: TEST/TEST/images/d_r_58_.jpg
tested: TEST/TEST/images/d_r_598_.jpg
tested: TEST/TEST/images/d_r_602_.jpg
tested: TEST/TEST/images/d_r_635_.jpg
tested: TEST/TEST/images/d_r_801_.jpg
tested: TEST/TEST/images/d_r_84_.jpg
tested: TEST/TEST/images/f_r_1033_.jpg
tested: TEST/TEST/images/f_r_1059_.jpg
tested: TEST/TEST/images/f_r_106_.jpg
tested: TEST/

KeyboardInterrupt: 

(3, 240, 320, 3)
(3, 240, 320, 5)


ValueError: Too many dimensions: 4 > 3.

In [4]:
#couleur fait maison 

#paths to masks 
samples_dir = "data/test/output/"
RO_dir = samples_dir + "RO/d_r_122_.bmp"
FB_dir = samples_dir + "FV/d_r_122_.bmp"
WR_dir = samples_dir + "WR/d_r_122_.bmp"
HD_dir = samples_dir + "HD/d_r_122_.bmp"
RI_dir = samples_dir + "RI/d_r_122_.bmp"


im_h, im_w = 240, 320


#création du mask final avec les couleurs
channel = 3
img_numpy = np.zeros((im_h, im_w, channel), dtype=np.uint8)

def assemblingMasks(img_name):

    classes = ["RO","FV","WR","HD","RI"]
    im_h, im_w = 240, 320
    channel = 3

    #création du mask final avec les couleurs
    
    img_numpy = np.zeros((im_h, im_w, channel), dtype=np.uint8)

    
    for p in classes:

        #get image and convert to np array
        mask = Image.open(samples_dir + p + "/" + img_name).resize((im_w, im_h))
        mask = np.array(mask)
        
        
        #set the color depending on the class
        
        for i in range(im_h):
            for j in range(im_w):
                if(mask[i,j] == 255):
                    #robots == red
                    if (p == "RO"):
                        img_numpy[i,j] = (255, 0, 0)
                    #Fish and vertebrates == yellow
                    elif (p == "FV"):
                        img_numpy[i,j] = (255, 255, 0) 
                    #Wrecks/ruins == cyan
                    elif (p == "WR"):
                        img_numpy[i,j] = (0, 255, 255) 
                    #Human divers == blue
                    elif (p == "HD"):
                        img_numpy[i,j] = (0, 0, 255) 
                    #Reefs and invertebrates
                    elif (p == "RI"):
                        img_numpy[i,j] = (255, 0, 255)         

            
    img = Image.fromarray(img_numpy, "RGB")
    img.show()

              
assemblingMasks("d_r_465_.bmp")

In [1]:
import tensorflowjs as tfjs

ModuleNotFoundError: No module named 'tensorflowjs'